In [1]:
#importing some files

import numpy as np
import pandas as pd

In [2]:
# Got some new data from Tript! Earlier Data is now redundant, I'll overwrite recipe_1.csv
recipe_x = pd.read_csv('data_new.csv')


/home/jushaan/Downloads/yes/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Dropping columns which were same in old data and new data

recipe_x.drop(['notes','ingredients','steps','author','cuisine','author','Unnamed: 0.1','Unnamed: 0'],axis = 1,inplace = True)

In [4]:
# New Recipe data spanning over all recipes, not only genius Kitchen and All recipes
recipe_x.head()

,Recipe_ID,calories,cook_time,prep_time,servings,title,total_time,url,continent,region,subregion
0,9757,248,0,0,8,Onion Tart,0,http://allrecipes.com/recipe/15629/onion-tart/,European,French,French
1,125883,327.9,0,0,4,Gnocchi,33,http://www.geniuskitchen.com/recipe/gnocchi-33...,European,Italian,Italian
2,52198,641.7,0,0,4,Chinese Firecracker Nachos II,26,http://www.geniuskitchen.com/recipe/chinese-fi...,Asian,Chinese,Chinese
3,48722,154.5,0,0,4,Fresh Pineapple With Coconut,15,http://www.geniuskitchen.com/recipe/fresh-pine...,African,African,African
4,80765,323.9,0,0,24,Horchata De Coco-- Yields 24 Servings,40,http://www.geniuskitchen.com/recipe/horchata-d...,Latin American,Mexican,Mexican


In [5]:
# UPDATING SERVINGS COLUMN
new_rec =pd.read_csv('newdata_jushaan.csv')

In [6]:
new_rec=new_rec[['Recipe_ID','servings']]
recipe_x.drop('servings',inplace = True,axis =1)
recipe_x = pd.merge(recipe_x,new_rec,on = 'Recipe_ID',how = 'left',sort = True)

In [7]:
recipe_x.head()

,Recipe_ID,calories,cook_time,prep_time,title,total_time,url,continent,region,subregion,servings
0,2610,196,30,15,Egyptian Lentil Soup,45,http://allrecipes.com/recipe/222661/egyptian-l...,African,Middle Eastern,Egyptian,4
1,2611,80,35,10,Egyptian Green Beans with Carrots,45,http://allrecipes.com/recipe/233456/egyptian-g...,African,Middle Eastern,Egyptian,4
2,2612,339,120,15,Egyptian Bamia,135,http://allrecipes.com/recipe/227986/egyptian-b...,African,Middle Eastern,Egyptian,4
3,2613,409,15,15,Magpie's Easy Falafel Cakes,60,http://allrecipes.com/recipe/143113/magpies-ea...,African,Middle Eastern,Egyptian,3
4,2614,45,5,20,Dukkah,25,http://allrecipes.com/recipe/79684/dukkah/,African,Middle Eastern,Egyptian,24


# Was Parsing the Json file, But Tript already did it and sent the file above

import json

with open('Raw_Recipes_Data.json') as json_file:
    data = json.load(json_file)
 

for i in range(len(data)):
    data[str(i+2610)].pop('ingredients',None)
    data[str(i+2610)].pop('author',None)
    data[str(i+2610)].pop('notes',None)
    data[str(i+2610)].pop('steps',None)

d = {int(k):v for k,v in data.items()}

df = pd.DataFrame.from_dict(d,orient='index')
df.reset_index(inplace = True)
df.rename({'index': 'recipe_no'},axis =1,inplace = True)
df.head()

df['source']=df['cuisine'].astype(str).apply(lambda x:x.split("/")[3])
df['region']=df['cuisine'].astype(str).apply(lambda x:x.split("/")[4])
df['country']=df['cuisine'].astype(str).apply(lambda x:x.split("/")[4])

df
data['2610']

 len(d)

In [8]:
# Creating a Dictionary for the source of the Data
dict_source = {'allrecipes':"AllRecipes",'tarladalal':'Tarladalal','thespruce':'TheSpruce','geniuskitchen':'Geniuskitchen','epicurious':'Epicurious','foodnetwork':'Foodnetwork','taste.com':'taste.com','mykoreankitchen':'MyKoreanKitchen','kraftrecipes':'KraftRecipes'}
# for every source in keys of dictionary locate which url of recipe_x contains
# the source and correspondingly put the value of that key in Source in the table

for source in dict_source.keys():
    recipe_x.loc[recipe_x.loc[:,'url'].str.contains(source),'Source']= dict_source[source]
        
    

In [9]:
# Check if all sources have been covered, should not have nan in list below

recipe_x.Source.unique()

array(['AllRecipes', 'Epicurious', 'Foodnetwork', 'Geniuskitchen',
       'KraftRecipes', 'MyKoreanKitchen', 'taste.com', 'TheSpruce'],
      dtype=object)

In [10]:
# sorting and exporting the file: known errors, Servings and Quantity has noise.

recipe_x.sort_values(['Recipe_ID'], inplace = True)

In [11]:
# Importing url of images of genius kitchen and all recipes
genius_img = pd.read_csv('geniuskitchen.csv',header = None)
allrecipes_img = pd.read_csv('allrecipesImages.csv',header = None)

In [12]:
# selecting relevant columns from file 
genius_img = genius_img[[0,5]].copy()

In [13]:
# renaming so I can concat with allrecipes_img
genius_img.rename({5:1},inplace = True,axis = 1)

In [14]:
allrecipes_img.drop_duplicates(inplace = True)

In [15]:
allrecipes_img.head() #15,477 rows are present

,0,1
0,2610,https://images.media-allrecipes.com/userphotos...
1,2611,https://images.media-allrecipes.com/userphotos...
2,2612,https://images.media-allrecipes.com/userphotos...
3,2613,https://images.media-allrecipes.com/userphotos...
4,2614,https://images.media-allrecipes.com/userphotos...


In [16]:
# Some allrecipes data do not have an image, to provide it a default image, I have added a url.

# list of id's for which we have data

q = list(allrecipes_img[0])

# as the list is sorted we are basically  just going through it and seeing which ones are missing and 
# Storing it in a list so we can append later

z = 0
n = []
for i in range(2610,18789):
    
    if i != q[z]:
        n.append(i)
    else:
        z+=1
        
# Now we'll add it to the dataframe!(albeit with a placeholder url)

for item in n:
    allrecipes_img.loc[allrecipes_img.index.max() + 1] = [item] + ['https://geniuskitchen.sndimg.com/gk/img/gk-shareGraphic.png']


In [17]:
# checking it's length if we have the right amount of rows 

len(list(allrecipes_img[0]))

16179

In [18]:
# Doing what we did above

genius_img.columns = range(genius_img.shape[1])
genius_img.drop_duplicates(inplace = True)
l = list(genius_img[0])
j=0
m =[]
for i in range(47200,149192):
    
    if i != l[j]:
        m.append(i)
        j-=1
    j+=1
for item in m:
    genius_img.loc[genius_img.index.max() + 1] = [item] + ['https://geniuskitchen.sndimg.com/gk/img/gk-shareGraphic.png']

# Again checking the length

print(len(list(genius_img[0])))

101992


In [19]:
# Concatenating the two tables into one
img = pd.concat([allrecipes_img,genius_img])
img.columns=['Recipe_ID','img_url']

In [20]:
img.head()

,Recipe_ID,img_url
0,2610,https://images.media-allrecipes.com/userphotos...
1,2611,https://images.media-allrecipes.com/userphotos...
2,2612,https://images.media-allrecipes.com/userphotos...
3,2613,https://images.media-allrecipes.com/userphotos...
4,2614,https://images.media-allrecipes.com/userphotos...


In [21]:
# Merging into one Recipes table
# if you change how to left, You will retain all values of recipes i.e. you won't drop nan's 

Recipes = pd.merge(recipe_x,img, how ='inner', on =['Recipe_ID'])

## Here we will add the nutrition information of recipes after taking them from DATA.ipynb

In [22]:
Recipes.head()

,Recipe_ID,calories,cook_time,prep_time,title,total_time,url,continent,region,subregion,servings,Source,img_url
0,2610,196,30,15,Egyptian Lentil Soup,45,http://allrecipes.com/recipe/222661/egyptian-l...,African,Middle Eastern,Egyptian,4,AllRecipes,https://images.media-allrecipes.com/userphotos...
1,2611,80,35,10,Egyptian Green Beans with Carrots,45,http://allrecipes.com/recipe/233456/egyptian-g...,African,Middle Eastern,Egyptian,4,AllRecipes,https://images.media-allrecipes.com/userphotos...
2,2612,339,120,15,Egyptian Bamia,135,http://allrecipes.com/recipe/227986/egyptian-b...,African,Middle Eastern,Egyptian,4,AllRecipes,https://images.media-allrecipes.com/userphotos...
3,2613,409,15,15,Magpie's Easy Falafel Cakes,60,http://allrecipes.com/recipe/143113/magpies-ea...,African,Middle Eastern,Egyptian,3,AllRecipes,https://images.media-allrecipes.com/userphotos...
4,2614,45,5,20,Dukkah,25,http://allrecipes.com/recipe/79684/dukkah/,African,Middle Eastern,Egyptian,24,AllRecipes,https://images.media-allrecipes.com/userphotos...


In [23]:
# Importing nutrition data, needs to be updated
rec_nut = pd.read_csv('../Nutrition_calc/nutrition_recipe_final.csv')
rec_nut


,recipe_no,Unnamed: 0,Alanine (g),"Alcohol, ethyl (g)",Arginine (g),Ash (g),Aspartic acid (g),Beta-sitosterol (mg),Betaine (mg),Caffeine (mg),...,Vitamin B-6 (mg),"Vitamin C, total ascorbic acid (mg)",Vitamin D (D2 + D3) (g),Vitamin D (IU),Vitamin D3 (cholecalciferol) (g),Vitamin E (alpha-tocopherol) (mg),"Vitamin E, added (mg)",Vitamin K (phylloquinone) (g),Water (g),"Zinc, Zn (mg)"
0,2610,2,0.0218,2.0164,0.0763,4.4216,6.5176,5.3541,90.4124,0.0000,...,0.7858,3.2640,1.8825,0.1669,0.1536,0.4840,69.6334,0.6366,0.2228,725.8722
1,2611,7,0.0426,0.0394,0.0326,4.1429,0.8265,0.1658,0.1280,0.1500,...,0.0000,0.0000,0.0000,1.4108,0.6622,4.3500,165.0190,442.8876,0.7094,0.0000
2,2612,14,0.0000,4.0039,0.0000,5.5042,6.3193,0.4877,0.0540,0.1500,...,0.4430,0.0000,0.4520,14.5400,7.7490,143.2641,429.2969,1.7808,0.0000,756.2702
3,2613,38,0.1068,2.8459,0.1393,6.8860,9.3267,7.4934,0.5288,3.5380,...,5.6089,13.5315,16.2540,0.0541,1.3332,0.3440,46.1657,59.0550,27.5636,26.1578
4,2614,26,1.2022,0.0000,3.2117,8.1360,2.7866,0.0000,1.2282,0.0000,...,0.1435,320.0000,22.5906,1.7195,0.5638,0.0000,4.2480,7.4212,1.0460,116.7200
5,2615,53,1.2349,0.0000,1.3548,12.0362,3.9576,0.0000,5.8560,0.0000,...,3.2533,590.7067,0.0000,3.4453,2.5556,0.1600,67.1303,922.4953,6.2939,175.1300
6,2616,53,0.8917,0.0955,1.7288,22.7393,1.9423,11.2924,0.0338,0.0000,...,0.2079,3.4000,0.0000,17.3057,0.2486,5.9428,1.7008,1108.1261,3.0573,2.6871
7,2617,50,0.8931,0.0158,1.9736,9.0563,1.9054,89.1587,0.3503,0.0000,...,9.7511,0.0000,40.8703,6.3253,0.3707,37.0287,15.6411,262.4622,48.3704,296.3888
8,2618,77,0.0783,2.4378,0.0565,7.1061,7.7413,6.6942,0.2022,0.1500,...,3.7290,3.1484,12.1905,1.1253,0.1614,260.9780,319.0737,57.2033,20.3712,24.6927
9,2619,118,0.2314,2.6566,0.4745,12.9052,9.3767,7.3064,0.0135,0.0000,...,4.0783,11.9940,9.5724,57.6234,0.3135,27.4094,260.6522,664.7134,28.1295,23.0285


In [24]:
rec_nut.rename({'recipe_no':'Recipe_id'},inplace =True, axis =1)

In [25]:
rec_nut['Recipe_id'] = rec_nut['Recipe_id'].astype(int)

In [26]:
l = list(rec_nut.columns)

In [27]:
Recipe_final = pd.merge(Recipes,rec_nut,left_on = 'Recipe_ID',right_on='Recipe_id',how = 'left')

In [28]:
Recipe_final.drop(['Recipe_id'],axis =1, inplace = True)

In [29]:
Recipe_final.sort_values('Recipe_ID', inplace = True)
Recipe_final.reset_index(inplace = True)
Recipe_final.drop(['index'],axis =1, inplace = True)

In [30]:
# Recipe_final.to_csv('./Recipes.csv')

In [31]:
# Recipe_final = pd.read_csv('Recipes.csv')
Recipe_final['Recipe_ID'].values

array([  2610,   2611,   2612, ..., 149189, 149190, 149191])

In [32]:
gkutensils =pd.read_csv('gkutensils.csv',header = None)
gkprocesses = pd.read_csv('gkprocesses.csv',header = None)
allRecuten = pd.read_csv('allRecipeUtensils.csv',header = None)
allRecproc = pd.read_csv('allRecipeProcesses.csv',header = None)


In [33]:
gkutensils.head()

,0,1
0,139755,jar||container
1,64905,NaN
2,50056,oven||bowl||jar||oven||pot||pan
3,51330,pot||pot
4,132984,bowl||baking||sheet||oven||baking||sheet||oven...


In [34]:
allRecuten.head()

,0,1
0,14563,saucepan||bowl
1,16421,pot||saucepan||popcorn||pan||large||paper||bag...
2,4927,oven||pan||bowl||bowl||pan||saucepan
3,2760,oven||baking||dish||skillet||plate||bowl||baki...
4,12978,oven||baking||dish||oven


In [35]:
uten = pd.concat([allRecuten,gkutensils])
uten.set_axis(['Recipe_ID','Utensils'],axis = 1, inplace = True)
proc = pd.concat([allRecproc,gkprocesses])
proc.set_axis(['Recipe_ID','Processes'],axis = 1,inplace =True)

In [36]:
uten['Recipe_ID'] = uten['Recipe_ID'].astype(int)
uten['Recipe_ID'].values
Recipe_final = pd.merge(Recipe_final,uten,on = 'Recipe_ID',how = 'left')

In [37]:
Recipe_final = pd.merge(Recipe_final,proc,on = 'Recipe_ID',how = 'left')

In [38]:
# New mapping

continent_mapping = pd.read_csv('recipe_full.csv')


In [39]:
Recipe_final.drop(['continent', 'region','subregion','calories','total_time'],axis = 1,inplace = True)

In [40]:
continent_mapping.drop(['Unnamed: 0','avg_val'],inplace = True,axis = 1)

In [41]:
continent_mapping.head()

,Recipe_ID,calories,total_time,continent,region,subregion
0,9757,248.0,0,European,French,French
1,125883,327.9,33,European,Italian,Italian
2,52198,641.7,26,Asian,Chinese and Mongolian,Chinese
3,48722,154.5,15,African,Rest Africa,Nigerian
4,80765,323.9,40,Latin American,Mexican,Mexican


In [42]:
Recipe_final = pd.merge(Recipe_final,continent_mapping,on = 'Recipe_ID',how = 'left')

In [43]:
Recipe_final

,Recipe_ID,cook_time,prep_time,title,url,servings,Source,img_url,Unnamed: 0,Alanine (g),...,Vitamin K (phylloquinone) (g),Water (g),"Zinc, Zn (mg)",Utensils,Processes,calories,total_time,continent,region,subregion
0,2610,30,15,Egyptian Lentil Soup,http://allrecipes.com/recipe/222661/egyptian-l...,4,AllRecipes,https://images.media-allrecipes.com/userphotos...,2.0,0.0218,...,0.6366,0.2228,725.8722,NaN,place||heat||cook||softened||remove||cool||ble...,196.0,45,African,Middle Eastern,Egyptian
1,2611,35,10,Egyptian Green Beans with Carrots,http://allrecipes.com/recipe/233456/egyptian-g...,4,AllRecipes,https://images.media-allrecipes.com/userphotos...,7.0,0.0426,...,442.8876,0.7094,0.0000,pot,heat||heat||cook||stir||stir||cook||stirring||...,80.0,45,African,Middle Eastern,Egyptian
2,2612,120,15,Egyptian Bamia,http://allrecipes.com/recipe/227986/egyptian-b...,4,AllRecipes,https://images.media-allrecipes.com/userphotos...,14.0,0.0000,...,1.7808,0.0000,756.2702,saucepan||oven||baking||dish||oven,heat||heat||cook||stir||mix||cook||stir||brown...,339.0,135,African,Middle Eastern,Egyptian
3,2613,15,15,Magpie's Easy Falafel Cakes,http://allrecipes.com/recipe/143113/magpies-ea...,3,AllRecipes,https://images.media-allrecipes.com/userphotos...,38.0,0.1068,...,59.0550,27.5636,26.1578,skillet||food||processor||bowl||cup||dish||tab...,cook||translucent||pulse||cornmeal||coriander|...,409.0,60,African,Middle Eastern,Egyptian
4,2614,5,20,Dukkah,http://allrecipes.com/recipe/79684/dukkah/,24,AllRecipes,https://images.media-allrecipes.com/userphotos...,26.0,1.2022,...,7.4212,1.0460,116.7200,oven||baking||sheet||skillet||bowl||skillet||p...,preheat||)||place||while||pour||fold||cover||r...,45.0,25,African,Middle Eastern,Egyptian
5,2615,30,30,Om Ali,http://allrecipes.com/recipe/19898/om-ali/,8,AllRecipes,https://images.media-allrecipes.com/userphotos...,53.0,1.2349,...,922.4953,6.2939,175.1300,oven||baking||dish||oven||oven||oven||bowl||su...,preheat||)||place||place||watch||remove||conti...,934.0,60,African,Middle Eastern,Egyptian
6,2616,25,10,Rice Pudding with Lemon Juice and Caramelized ...,http://allrecipes.com/recipe/231759/rice-puddi...,4,AllRecipes,https://images.media-allrecipes.com/images/795...,53.0,0.8917,...,1108.1261,3.0573,2.6871,saucepan||skillet||bowl,heat||heat||add||simmer||tender||heat||heat||c...,299.0,35,African,Middle Eastern,Egyptian
7,2617,30,20,Couscous with Olives and Sun-Dried Tomato,http://allrecipes.com/recipe/232210/couscous-w...,4,AllRecipes,https://images.media-allrecipes.com/userphotos...,50.0,0.8931,...,262.4622,48.3704,296.3888,saucepan||skillet||saucepan||bowl,boil||stir||mix||reduce||heat||simmer||heat||h...,528.0,50,African,Middle Eastern,Egyptian
8,2618,10,10,Fava Bean Breakfast Spread,http://allrecipes.com/recipe/17596/fava-bean-b...,6,AllRecipes,https://images.media-allrecipes.com/userphotos...,77.0,0.0783,...,57.2033,20.3712,24.6927,pot,pour||boil||mix||add||boil||serve||pita,101.0,20,African,Middle Eastern,Egyptian
9,2619,60,20,Koshary,http://allrecipes.com/recipe/214832/koshary/,8,AllRecipes,https://images.media-allrecipes.com/userphotos...,118.0,0.2314,...,664.7134,28.1295,23.0285,bag||container||pot||pot||saucepan||pot||pot||...,combine||coriander||store||dish||boil||cook||c...,498.0,80,African,Middle Eastern,Egyptian


In [44]:
Recipe_final.index.name = 'Recipe_index'

In [45]:
Recipe_final.rename({'Recipe_ID':'Recipe_id','calories':'Calories','title':'Recipe_title','continent':'Continent','region':'Region','subregion':'Sub_region'},axis =1,inplace = True)

In [46]:
Recipe_final[l] = Recipe_final[l].round(4)

In [47]:
Recipe_final.to_csv('./Recipes(5).csv',float_format='%.4f')

##changing the processes and utensils

In [48]:
utensils = ["bowl","pan","oven","skillet","pot","saucepan","cup","dish","sheet","processor","fork","knife","plate","tablespoon","mixer","container","spatula","spoon","cooker","whisk","board","microwave","platter","sieve","boiler","jar","casserole","masher","skewer","cutter","frypan","strainer","stockpot","ladle","shaker","crockpot","peeler","crock","tray","blender","saucepot","basket","teaspoon","saucer","jug","ramekin","mug","bottle","kettle","beater","scoop","foil","paddle","blade","corer","processer","fryer","frying-pan","grinder","saucpan","box","pestle","shell","sauceboat","disc","suacepan","marzipan","steamer","basin"]
processes = ["absorbed" ,"add" ,"al dente" ,"arrange" ,"assemble" ,"bake" ,"barbecue" ,"baste" ,"beat" ,"blanch" ,"blend" ,"blitz" ,"blot" ,"boil" ,"braise" ,"break" ,"broil" ,"broil" ,"brush" ,"bubble" ,"bubbling" ,"burn" ,"butter" ,"caramelize" ,"caramelized" ,"carve" ,"char" ,"charred" ,"check" ,"chill" ,"chop" ,"clean" ,"coat" ,"combine" ,"condensed" ,"cook" ,"cool" ,"cover" ,"cream" ,"crimp" ,"crockpot" ,"crumble" ,"crush" ,"curdle" ,"cut" ,"dash" ,"decorate" ,"deflate" ,"defrost" ,"deglaze" ,"devein" ,"dice" ,"dip" ,"dissolve" ,"distribute" ,"divide" ,"dollop" ,"drain" ,"dredge" ,"dress" ,"dressing" ,"drip" ,"dripping" ,"drizzle" ,"drop" ,"dry" ,"dump" ,"dust" ,"evaporate" ,"evaporated" ,"fill" ,"flake" ,"flatten" ,"flavoring" ,"flip" ,"floured" ,"foam" ,"foil" ,"fold" ,"freezing" ,"frost" ,"fry" ,"garnish" ,"glaze" ,"grate" ,"grease" ,"griddle" ,"grind" ,"handle" ,"heat" ,"hold" ,"ice" ,"immerse" ,"insert" ,"invert" ,"knead" ,"ladle" ,"lard" ,"marinade" ,"marinate" ,"mash" ,"massage" ,"measure" ,"meld" ,"melt" ,"mince" ,"minced" ,"mix" ,"moist" ,"moisten" ,"mould" ,"mound" ,"move" ,"muddle" ,"note" ,"open" ,"overcook" ,"overmix" ,"pack" ,"pan-fry" ,"parboil" ,"pat" ,"peel" ,"pick" ,"pierce" ,"place" ,"poach" ,"poke" ,"pop" ,"pour" ,"powdered" ,"pre-heat" ,"preheat" ,"prepare" ,"presentation" ,"press" ,"pressure" ,"prick" ,"Process" ,"process" ,"pull" ,"punch" ,"puree" ,"push" ,"put" ,"raise" ,"reduce" ,"refrigerate" ,"reheat" ,"remove" ,"repeat" ,"replace" ,"reserve" ,"rest" ,"rinse" ,"rise" ,"rising" ,"roast" ,"roll" ,"rotate" ,"roux" ,"rub" ,"saute" ,"save" ,"scald" ,"scatter" ,"scoop" ,"scorching" ,"scramble" ,"scrape" ,"scrub" ,"seal" ,"sear" ,"season" ,"seasoning" ,"select" ,"separate" ,"serve" ,"set" ,"settle" ,"shake" ,"shape" ,"shred" ,"sieve" ,"sift" ,"simmer" ,"sit" ,"sizzle" ,"skim" ,"skin" ,"slash" ,"slather" ,"slice" ,"slit" ,"smash" ,"smear" ,"smoke" ,"smoking" ,"smooth" ,"snap" ,"soak" ,"soften" ,"sort" ,"splash" ,"split" ,"splutter" ,"spray" ,"spread" ,"sprinkle" ,"sprout" ,"square" ,"squash" ,"squeeze" ,"stack" ,"stand" ,"start" ,"steam" ,"steep" ,"sterilize" ,"stew" ,"stir" ,"stir-fry" ,"stirring" ,"stop" ,"store" ,"strain" ,"stream" ,"stretch" ,"strip" ,"stuff" ,"style" ,"submerge" ,"sweat" ,"sweeten" ,"swirl" ,"take" ,"taste" ,"tear" ,"test" ,"thaw" ,"thicken" ,"thread" ,"throw" ,"tie" ,"tilt" ,"toast" ,"top" ,"toss" ,"touch" ,"transfer" ,"trim" ,"tuck" ,"turn" ,"twist" ,"uncover" ,"uncovered" ,"unfold" ,"unmold" ,"unroll" ,"unwrap" ,"wait" ,"warm" ,"wash" ,"wet" ,"whip" ,"whirl" ,"whisk" ,"wilt" ,"wipe" ,"wok" ,"wrap" ,"yield" ,"zest"]

In [49]:
import csv
with open('Recipes(6).csv','w') as csvOutput:
    writer =csv.writer(csvOutput,delimiter=',',quotechar='"')
    with open('Recipes(5).csv','rt',errors ='ignore') as f:
        data = csv.reader(f)
        header = next(data)
        y = writer.writerow(header)
        for row in data:
            processes_new=[]
            utensils_new =[]
            s=''
            p=''
            processes_temp = row[-6].split('||')
            for i in processes_temp:
                if i in processes:
                    processes_new.append(i)
            s='||'.join(processes_new)
            row[-6]=s
            utensils_temp = row[-7].split('||')
            for i in utensils_temp:
                if i in utensils:
                    utensils_new.append(i)
            p='||'.join(utensils_new)
            row[-7]=p
            y = writer.writerow(row)
            
        

In [50]:
r = pd.read_csv('Recipes(6).csv')

/home/jushaan/Downloads/yes/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (2,3,6,161) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:
r = r[['Recipe_id','Calories','cook_time','prep_time','servings','Recipe_title', 'total_time','url','Region',
'Sub_region','Continent','Source','img_url','Carbohydrate, by difference (g)','Energy (kcal)','Protein (g)','Total lipid (fat) (g)','Utensils',
 'Processes' ]]

In [52]:
AR_GK_diet= pd.read_csv('dietary_styles.csv',header = None)
AR_GK_diet.columns = ['Recipe_id','vegan','pescetarian','ovo_vegetarian','lacto_vegetarian','ovo_lacto_vegetarian']


In [53]:
AR_GK_diet = AR_GK_diet[AR_GK_diet['Recipe_id'] != 'recipe_no'].copy()

In [54]:
AR_GK_diet['Recipe_id']=AR_GK_diet['Recipe_id'].astype(int)


In [55]:
y = pd.merge(r,AR_GK_diet,how = 'left',on = 'Recipe_id')
y.replace({'servings':'Servings'},inplace = True)

In [56]:
y.to_csv('./Recipe_small.csv',float_format='%.4f',index = None)